What are the keys?

Args.env_id <== facet
Args.seed <== group by.

Table View

In [1]:
%load_ext autoreload
%autoreload 2

from plan2vec_experiments import thunk
# from multiprocessing import Pool
# 
# pool = Pool(10)

In [2]:
from plan2vec_experiments import RUN
from ml_logger import logger

logger.configure(RUN.server, prefix="geyang/plan2vec/2019/12-16/analysis/local-metric-analysis/analysis")

with logger.PrefixContext(logger.prefix, ".."):
    print(logger.prefix)
    exps = logger.glob("**/parameters.pkl")

creating new logging client... ✓ done
Dashboard: http://localhost:3001/geyang/plan2vec/2019/12-16/analysis/local-metric-analysis/analysis
Log_directory: http://54.71.92.65:8081
══════════════════════════════════════════
        run         
────────────────────┬─────────────────────
     createTime     │ 2019-12-17 00:21:45.545505+00:00
       prefix       │ geyang/plan2vec/2019/12-16/analysis/local-metric-analysis/analysis
════════════════════╧═════════════════════
geyang/plan2vec/2019/12-16/analysis/local-metric-analysis


In [3]:
local_metric_exps = [p for p in exps if not p.startswith('analysis')]
print(*local_metric_exps, sep="\n")

all_local_metric/02.26/rope/K-(1)/34.413608/parameters.pkl
all_local_metric/02.26/rope/K-(1)/30.770096/parameters.pkl
all_local_metric/02.26/rope/K-(1)/28.883938/parameters.pkl
all_local_metric/02.26/rope/K-(1)/32.590673/parameters.pkl
all_local_metric/02.26/rope/K-(1)/36.201837/parameters.pkl
all_local_metric/02.26/streetlearn/ResNet18L2/lr-(1e-05)/41.078741/parameters.pkl
all_local_metric/02.26/streetlearn/ResNet18L2/lr-(1e-05)/43.084514/parameters.pkl
all_local_metric/02.26/streetlearn/ResNet18L2/lr-(1e-05)/46.871195/parameters.pkl
all_local_metric/02.26/streetlearn/ResNet18L2/lr-(1e-05)/45.063410/parameters.pkl
all_local_metric/02.26/streetlearn/ResNet18L2/lr-(1e-05)/39.206687/parameters.pkl
all_local_metric/12.28/streetlearn/LocalMetricConvDeep/lr-(1e-05)/17.656891/parameters.pkl
all_local_metric/12.28/streetlearn/ResNet18L2/lr-(1e-05)/24.711204/parameters.pkl
all_local_metric/11.55/rope/K-(1)/33.474485/parameters.pkl
all_local_metric/11.55/rope/K-(1)/37.085705/parameters.pkl
all_

In [12]:
from os.path import dirname
from tqdm import tqdm

logger.summary_cache.clear()

for exp_path in tqdm(local_metric_exps):
    with logger.PrefixContext(logger.prefix, "..", dirname(exp_path)):
        try:
            env_id, local_metric = logger.get_parameters('Args.env_id', "Args.local_metric", default=None, silent=True)
            acc_15, acc_11, eval_acc_15, eval_acc_11 = \
                logger.get_metrics("accuracy-1.5/mean", "accuracy-1.1/mean", 
                                   "eval/accuracy-1.5/mean", "eval/accuracy-1.1/mean", 
                                   default=None)
            logger.store_metrics({"prefix":logger.prefix, 
                                  "accuracy-1.1": acc_11[-20:].to_numpy(),
                                  "accuracy-1.5": acc_15[-20:].to_numpy(),
                                  "eval/accuracy-1.1": eval_acc_11[-20:].to_numpy(),
                                  "eval/accuracy-1.5": eval_acc_15[-20:].to_numpy()
                                  },  env_id=env_id)
            # logger.peek_stored_metrics(len=3)
        except Exception as e:
            print(exp_path)
            print(e)
            pass

100%|██████████| 32/32 [00:58<00:00,  1.83s/it]


all_local_metric/12.28/streetlearn/ResNet18L2/lr-(1e-05)/24.711204/parameters.pkl
'NoneType' object is not subscriptable
all_local_metric/11.55/img_maze/回MazeDiscreteImgIdLess-v0/02.755501/parameters.pkl
'NoneType' object is not subscriptable
all_local_metric/11.55/img_maze/GoalMassDiscreteImgIdLess-v0/01.044009/parameters.pkl
'NoneType' object is not subscriptable


In [6]:
logger.peek_stored_metrics("env_id", "prefix", len=None)

           env_id           |                                                               prefix                                                                
━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
            rope            |                geyang/plan2vec/2019/12-16/analysis/local-metric-analysis/all_local_metric/02.26/rope/K-(1)/34.413608                
            rope            |                geyang/plan2vec/2019/12-16/analysis/local-metric-analysis/all_local_metric/02.26/rope/K-(1)/30.770096                
            rope            |                geyang/plan2vec/2019/12-16/analysis/local-metric-analysis/all_local_metric/02.26/rope/K-(1)/28.883938                
            rope            |                geyang/plan2vec/2019/12-16/analysis/local-metric-analysis/all_local_metric/02.26/rope/K-(1)/32.590673                
            rope      

In [13]:
import numpy as np
import pandas as pd
dataframe = pd.DataFrame(logger.summary_cache.data)
grouped = dataframe.groupby('env_id')
a= grouped['accuracy-1.5'].agg([
    ["accuracy-1.5", lambda c: np.nanmean(np.concatenate(c.to_numpy()))], 
    ["±25%", lambda c: np.nanquantile(np.concatenate(c.to_numpy()), 0.75) 
                     - np.nanquantile(np.concatenate(c.to_numpy()), 0.25)]
    ])
b=grouped['accuracy-1.1'].agg([
    ["accuracy-1.1", lambda c: np.nanmean(np.concatenate(c.to_numpy()))], 
    ["1.1-±25%", lambda c: np.nanquantile(np.concatenate(c.to_numpy()), 0.75) 
                     - np.nanquantile(np.concatenate(c.to_numpy()), 0.25)]
    ])
c=grouped['eval/accuracy-1.5'].agg([
    ["eval/accuracy-1.5", lambda c: np.nanmean(np.concatenate(c.to_numpy()))], 
    ["eval/1.5-±25%", lambda c: np.nanquantile(np.concatenate(c.to_numpy()), 0.75) 
                     - np.nanquantile(np.concatenate(c.to_numpy()), 0.25)]
    ])
d=grouped['eval/accuracy-1.1'].agg([
    ["eval/accuracy-1.1", lambda c: np.nanmean(np.concatenate(c.to_numpy()))], 
    ["eval/1.1-±25%", lambda c: np.nanquantile(np.concatenate(c.to_numpy()), 0.75) 
                     - np.nanquantile(np.concatenate(c.to_numpy()), 0.25)]
    ])

_= pd.merge(a, b, on=['env_id'], how='inner')
_= pd.merge(_, c, on=['env_id'], how='inner')
account = pd.merge(_, d, on=['env_id'], how='inner')
print(account)

                              accuracy-1.5      ±25%  accuracy-1.1  1.1-±25%  \
env_id                                                                         
CMazeDiscreteImgIdLess-v0         0.999358  0.001167      0.946526  0.021667   
GoalMassDiscreteImgIdLess-v0      0.999750  0.000333      0.945531  0.020000   
rope                              0.966413  0.007422      0.969112  0.007422   
streetlearn                       0.992252  0.004219      0.971372  0.014063   
回MazeDiscreteImgIdLess-v0         0.999618  0.000000      0.957031  0.016333   

                              eval/accuracy-1.5  eval/1.5-±25%  \
env_id                                                           
CMazeDiscreteImgIdLess-v0              0.974432       0.004056   
GoalMassDiscreteImgIdLess-v0           0.977966       0.003194   
rope                                   0.923977       0.014620   
streetlearn                            0.993113       0.000920   
回MazeDiscreteImgIdLess-v0              0.97

In [ ]:
@thunk
def grab_accuracy_table():
    from ml_logger import logger

    with logger.PrefixContext("/geyang/plan2vec/2019/12-15/analysis/local-metric-analysis"):
        exps = logger.glob("**/parameters.pkl")
        for path in exps:
            if path.startswith('analysis'):
                continue
            print(path)
            try:
                env_id = logger.get_parameters("Args.env_id", path=path)
                accuracy = logger.get_metrics("accuracy-1.5/mean", path=path.replace('parameters', 'metrics'))
                tail = accuracy[-100:]
                logger.log(env_id=env_id, accuracy=tail.mean(), pm25=tail.quantile(.25), pm75=tail.quantile(0.75),
                           flush=True)
            except Exception as e:
                print(">>>>", e)
                pass



